In [1]:

import __init__
from utils.lifelines_preprocess import surv_functions
import os

wd = "/exports/reum/dvanderbijl/"
data_dir = "/exports/reum/nsteinz/data/data_03_12_22/"
results_dir = "/exports/reum/nsteinz/results/"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors, KDTree, kneighbors_graph
from sklearn.metrics import pairwise_distances

from lifelines import CoxPHFitter

import seaborn as sns

imports worked


In [2]:
subproject_name = "trajectories"

wd = "../../../"
project_path = os.path.realpath(wd)
print(project_path)

raw_data_dir = project_path+"/raw_data/"
print(raw_data_dir)


data_dir = project_path+f"/{subproject_name}"+"/data/"
print(data_dir)

results_dir = project_path+f"/{subproject_name}"+"/output"
print(results_dir)


/exports/reum/nsteinz/Project_trajectories
/exports/reum/nsteinz/Project_trajectories/raw_data/
/exports/reum/nsteinz/Project_trajectories/trajectories/data/
/exports/reum/nsteinz/Project_trajectories/trajectories/output


In [7]:
dmards = ['ABATACEPT',
       'ADALIMUMAB', 'AZATHIOPRINE', 'BARICITINIB', 'CERTOLIZUMAB_PEGOL',
       'CYCLOPHOSPHAMIDE', 'ETANERCEPT', 'GOLIMUMAB', 'HYDROXYCHLOROQUINE',
       'INFLIXIMAB', 'LEFLUNOMIDE', 'METHOTREXATE',
       'RITUXIMAB', 'SODIUM_AUROTHIOMALATE', 'SULFASALAZINE',
       'TOCILIZUMAB', 'TOFACITINIB']
dmard_types = {
    "TNF_b_dmards" : ['ETANERCEPT', 'GOLIMUMAB','INFLIXIMAB','CERTOLIZUMAB_PEGOL', 'ADALIMUMAB'],
    "IL6_b_dmards" : ['CERTOLIZUMAB_PEGOL'],
    "IL1_b_dmards" : ['TOCILIZUMAB'],
    "tcell_b_dmards" : ['ABATACEPT'],
    "cd20_b_dmards" : ['RITUXIMAB'],
    "cs_dmards" : ["METHOTREXATE",'HYDROXYCHLOROQUINE','LEFLUNOMIDE', 'SULFASALAZINE'],
    "jak_dmard" : ['TOFACITINIB', 'BARICITINIB'],
    "XO_damrd" : ['AZATHIOPRINE'],
    "OP_dmard" : ["CYCLOPHOSPHAMIDE"],
    "predni":   ['PREDNISOLONE',"METHYLPREDNISOLONE"],
}
predni=   ['PREDNISOLONE',"METHYLPREDNISOLONE"]
#predni =  ['PREDNISOLONE',"METHYLPREDNISOLONE"]

In [29]:
leiden_features = [
            'TJC_28',
            'SJC_28',
            'ESR',
            'leukocytes',
            "hb_corrected",
            'thrombocytes',
            'MCV',
           ]

features_o = [f"{x}_o" for x in leiden_features]
leiden = pd.read_csv(data_dir+ "predictie_data.csv",)
leiden
leiden.columns

Index(['TJC_28', 'SJC_28', 'ESR', 'leukocytes', 'hb_corrected', 'MCV',
       'thrombocytes', 'TJC_28_o', 'SJC_28_o', 'ESR_o', 'leukocytes_o',
       'hb_corrected_o', 'MCV_o', 'thrombocytes_o', 'das28_n', 'das28',
       'das44', 'SJC_44_o', 'TJC_44_o', 'aCCP positive', 'RF positive',
       'label', 'days', 'days_nlp', 'PATNR', 'DATUM', 'Age', 'sex',
       'dmard_usage', 'symptomDuration', 'days_DMARD', 'Hemoglobine',
       'sequence', 'Age_n', 'remision', 'ABATACEPT', 'ADALIMUMAB', 'ANAKINRA',
       'SODIUM_AUROTHIOMALATE', 'AZATHIOPRINE', 'BARICITINIB',
       'CERTOLIZUMAB_PEGOL', 'CICLOSPORINE', 'CYCLOPHOSPHAMIDE', 'ETANERCEPT',
       'GOLIMUMAB', 'HYDROXYCHLOROQUINE', 'INFLIXIMAB', 'LEFLUNOMIDE',
       'METHOTREXATE', 'SULFASALAZINE', 'TOCILIZUMAB', 'TOFACITINIB',
       'METHYLPREDNISOLONE', 'PREDNISOLONE', 'TNF_b_dmards', 'IL6_b_dmards',
       'IL1_b_dmards', 'tcell_b_dmards', 'cs_dmards', 'jak_dmard', 'XO_damrd',
       'OP_dmard', 'TC_dmard', 'gld_dmard', 'count', 'no_

In [14]:
tacera_features = [
            'TJC_28', 
            'SJC_28',
            'ESR',
            'leukocytes',        
            'hemoglobin',
            'thrombocytes',
           ]
tacera = pd.read_csv(data_dir+"tacera_cluster_data.csv")
tacera.columns

Index(['index', 'Subject ID', 'months', 'Study', 'Age', 'Race', 'Sex',
       'Weight', 'Date of Assessment', 'Date of symptom onset', 'Final Score',
       'Unnamed: 10', 'acpa', 'RF', 'ACPA positive SCR',
       'Rheum Factor positive SCR', 'Basophils', 'CRP Value (mg/L)',
       'DAS-28 CRP', 'DAS28', 'ESR', 'Eosinophils', 'GL', 'HB', 'Lymphocytes',
       'Monocytes', 'Neutrophils', 'thrombocytes', 'SJC_28', 'TJC_28',
       'leukocytes', 'Adalimumab', 'Certolizumab', 'Dexamethasone',
       'Etanercept', 'Folic Acid', 'Golimumab', 'Hydroxychloroquine',
       'Leflunomide', 'Methotrexate', 'Methylprednisolone',
       'Methylprednisolone Acetate', 'Prednisolone', 'Sulfasalazine',
       'Tocilizumab', 'Triamcinolone (Kenalog)', 'days', 'seropos',
       'HB_corrected', 'hemoglobin', 'das_28_c', 'visit_count', 'predni',
       'm_dmards', 'dmard_usage', 'label', 'TJC_28_n', 'SJC_28_n', 'ESR_n',
       'leukocytes_n', 'hemoglobin_n', 'thrombocytes_n', 'das28_n', 'das28',
       'aCC

In [10]:

def print_latex_table_EHR(dataset, 
                          groupby="Subject ID",
                          query = "label==@j" , 
                          type_ = "trajectory", 
                          volgorde= None,
                         features_median = [],
                          features_mean= [],
                          features_add = ["dmard_usage","RF", "acpa", "Sex"]
                         ):
    if query == None:
        query = 'ilevel_0 in ilevel_0'
    print("Features", end ="")
    for i in volgorde:
        print(" & ", end="")
        print(type_,i, end="")
    print("\\\\\\hline")

    for j in volgorde:

        z = (dataset.query(query)
                        .groupby(groupby)
                        )
        print(f"n= &{z.head(1).count().iloc[1]}&")
        visits = z.count().iloc[:,1]
        print(f"nr of visits (IQR)& {visits.median()} ({float(round(visits.quantile(q=0.25),1))} - {float(round(visits.quantile(q=0.75),1))})&", end="")
    print("\\\\")
    for i in features_median:
        print(i,"$\space$ (IQR) $\space\dagger$ ", end= "")
        for j in volgorde:
            print(" & ", end="")
            z = (dataset.query(query)
                        .groupby(groupby)
                        .head(1)[i])
            print(f"{float(round(z.median(),1))} ({float(round(z.quantile(q=0.25),1))} - {float(round(z.quantile(q=0.75),1))})", end="")
        print("\\\\")
    for i in features_mean:
        print(i,"$\space$ (±IQR) $\space\dagger$ ", end= "")
        for j in volgorde:
            print(" & ", end="")
            z = (dataset.query(query)
                        .groupby(groupby)
                        .head(1)[i])
            print(f"{float(round(z.mean(),2))} ({float(round(z.std(),2))})", end="")
        print("\\\\")
    for i in features_add:
        print(i,"(\\%)", end= "")
        for j in volgorde:
            print(" & ", end="")
            z = (dataset.query(query)
                        .groupby(groupby)
                        .head(1)[i])
            print(f"{int(round(z.sum(),0))} ({round(z.mean()*100, 1)})", end="")

        print("\\\\")



In [55]:

baseline_leiden = leiden.sort_values("days").groupby("PATNR").head(1).reset_index()
print_latex_table_EHR(baseline_leiden, groupby="PATNR", query=None,type_="", volgorde =[5], features_median= features_o + ["das28"], features_mean=  ["Age", "seropos",], features_add=  ["dmard_usage","RF positive", "aCCP positive", "sex"] )

Features &  5\\\hline
n= &1237&
nr of visits (IQR)& 1.0 (1.0 - 1.0)&\\
TJC_28_o $\space$ (IQR) $\space\dagger$  & 4.0 (2.0 - 8.0)\\
SJC_28_o $\space$ (IQR) $\space\dagger$  & 3.0 (1.0 - 7.0)\\
ESR_o $\space$ (IQR) $\space\dagger$  & 28.0 (11.0 - 45.0)\\
leukocytes_o $\space$ (IQR) $\space\dagger$  & 7.9 (6.3 - 9.6)\\
hb_corrected_o $\space$ (IQR) $\space\dagger$  & 8.4 (7.8 - 8.9)\\
thrombocytes_o $\space$ (IQR) $\space\dagger$  & 290.0 (246.0 - 352.0)\\
MCV_o $\space$ (IQR) $\space\dagger$  & 90.0 (87.0 - 93.0)\\
das28 $\space$ (IQR) $\space\dagger$  & 4.3 (3.3 - 5.3)\\
Age $\space$ (±IQR) $\space\dagger$  & 59.82 (14.73)\\
seropos $\space$ (±IQR) $\space\dagger$  & 0.55 (0.5)\\
dmard_usage (\%) & 970 (78.4)\\
RF positive (\%) & 570 (46.1)\\
aCCP positive (\%) & 571 (46.2)\\
sex (\%) & 797 (64.4)\\


In [49]:

last_leiden = leiden.sort_values("days").groupby("PATNR").tail(1).reset_index()
print_latex_table_EHR(last_leiden, groupby="PATNR", query=None,type_="", volgorde =[5], features_median= features_o + ["das28"], features_mean=  ["Age", "seropos",], features_add=  ["dmard_usage","RF positive", "aCCP positive", "sex"] )

Features &  5\\\hline
n= &1237&
nr of visits (IQR)& 1.0 (1.0 - 1.0)&\\
TJC_28_o $\space$ (IQR) $\space\dagger$  & 1.0 (0.0 - 3.0)\\
SJC_28_o $\space$ (IQR) $\space\dagger$  & 0.0 (0.0 - 2.0)\\
ESR_o $\space$ (IQR) $\space\dagger$  & 14.0 (6.0 - 29.0)\\
leukocytes_o $\space$ (IQR) $\space\dagger$  & 7.2 (6.0 - 8.8)\\
hb_corrected_o $\space$ (IQR) $\space\dagger$  & 8.5 (8.0 - 9.0)\\
thrombocytes_o $\space$ (IQR) $\space\dagger$  & 264.0 (225.0 - 314.0)\\
MCV_o $\space$ (IQR) $\space\dagger$  & 92.0 (89.0 - 96.0)\\
das28 $\space$ (IQR) $\space\dagger$  & 2.9 (2.1 - 3.9)\\
Age $\space$ (±IQR) $\space\dagger$  & 59.82 (14.73)\\
seropos $\space$ (±IQR) $\space\dagger$  & 0.55 (0.5)\\
dmard_usage (\%) & 983 (79.5)\\
RF positive (\%) & 570 (46.1)\\
aCCP positive (\%) & 571 (46.2)\\
sex (\%) & 797 (64.4)\\


In [50]:

baseline_tacera = tacera.sort_values("days").groupby("Subject ID").head(1).reset_index()
print_latex_table_EHR(baseline_tacera, query=None,type_="", volgorde =[5], features_median= tacera_features + ["das_28_c"], features_mean=  ["Age", "seropos",] )

Features &  5\\\hline
n= &243&
nr of visits (IQR)& 1.0 (1.0 - 1.0)&\\
TJC_28 $\space$ (IQR) $\space\dagger$  & 9.0 (4.0 - 14.0)\\
SJC_28 $\space$ (IQR) $\space\dagger$  & 6.0 (3.0 - 9.5)\\
ESR $\space$ (IQR) $\space\dagger$  & 27.0 (12.0 - 42.5)\\
leukocytes $\space$ (IQR) $\space\dagger$  & 7.8 (6.6 - 9.2)\\
hemoglobin $\space$ (IQR) $\space\dagger$  & 8.5 (7.9 - 8.9)\\
thrombocytes $\space$ (IQR) $\space\dagger$  & 293.0 (255.0 - 365.0)\\
das_28_c $\space$ (IQR) $\space\dagger$  & 5.0 (4.1 - 5.8)\\
Age $\space$ (±IQR) $\space\dagger$  & 52.95 (14.92)\\
seropos $\space$ (±IQR) $\space\dagger$  & 1.0 (0.0)\\
dmard_usage (\%) & 223 (91.8)\\
RF (\%) & 223 (91.8)\\
acpa (\%) & 207 (85.5)\\
Sex (\%) & 177 (72.8)\\


In [51]:

last_tacera = tacera.sort_values("days").groupby("Subject ID").tail(1).reset_index()
print_latex_table_EHR(last_tacera, query=None,type_="", volgorde =[5], features_median= tacera_features + ["das_28_c"], features_mean=  ["Age", "seropos",] )

Features &  5\\\hline
n= &243&
nr of visits (IQR)& 1.0 (1.0 - 1.0)&\\
TJC_28 $\space$ (IQR) $\space\dagger$  & 2.0 (0.0 - 5.0)\\
SJC_28 $\space$ (IQR) $\space\dagger$  & 1.0 (0.0 - 3.0)\\
ESR $\space$ (IQR) $\space\dagger$  & 12.0 (6.0 - 24.0)\\
leukocytes $\space$ (IQR) $\space\dagger$  & 6.3 (5.1 - 7.8)\\
hemoglobin $\space$ (IQR) $\space\dagger$  & 8.5 (8.0 - 8.9)\\
thrombocytes $\space$ (IQR) $\space\dagger$  & 266.5 (225.5 - 312.0)\\
das_28_c $\space$ (IQR) $\space\dagger$  & 3.0 (2.0 - 3.9)\\
Age $\space$ (±IQR) $\space\dagger$  & 52.95 (14.92)\\
seropos $\space$ (±IQR) $\space\dagger$  & 1.0 (0.0)\\
dmard_usage (\%) & 229 (94.2)\\
RF (\%) & 223 (91.8)\\
acpa (\%) & 207 (85.5)\\
Sex (\%) & 177 (72.8)\\


In [59]:
from scipy.stats import ttest_ind
for l,t in zip(features_o,tacera_features):
    print(l,t)
    print(baseline_leiden[l].mean(),baseline_tacera[t].mean() )
    print(baseline_leiden[l].std(),baseline_tacera[t].std() )
    print(ttest_ind(baseline_leiden[l],baseline_tacera[t], equal_var= False))

TJC_28_o TJC_28
5.592562651576395 9.748971193415638
5.2427057955384155 6.727176670935341
TtestResult(statistic=-9.103598001064311, pvalue=1.217709953586865e-17, df=302.3489370344904)
SJC_28_o SJC_28
4.518189167340339 6.979423868312757
4.757283697084143 5.1373277933944745
TtestResult(statistic=-6.908967762174033, pvalue=2.5402578019055688e-11, df=328.57311408707733)
ESR_o ESR
32.98059822150364 30.650205761316872
28.20673268738346 24.041739487000378
TtestResult(statistic=1.3405890952334805, pvalue=0.18084426877026538, df=385.0566988450104)
leukocytes_o leukocytes
8.153257887693018 7.955349794238683
2.799441313511057 2.239150744104633
TtestResult(statistic=1.2051381724266366, pvalue=0.22885203435496293, df=405.9360571898807)
hb_corrected_o hemoglobin
8.093303714406295 8.357488888888888
1.39163438838891 0.8883589266034493
TtestResult(statistic=-3.8079284338560373, pvalue=0.00015724773154494315, df=508.42678021594173)
thrombocytes_o thrombocytes
307.51131770412286 316.25925925925924
95.2222

In [62]:
from scipy.stats import ttest_ind
for l,t in zip(features_o,tacera_features):
    print(l,t)
    print(last_leiden[l].mean(),last_tacera[t].mean() )
    print(last_leiden[l].std(),last_tacera[t].std() )
    print(ttest_ind(last_leiden[l].dropna(),last_tacera[t].dropna() , equal_var= False))

TJC_28_o TJC_28
2.5157639450282945 3.5761316872427984
3.6873068599599663 5.235504018687619
TtestResult(statistic=-3.013774234895384, pvalue=0.002807378103688344, df=290.91793656519434)
SJC_28_o SJC_28
1.7299919159256265 1.7818930041152263
2.8126242942573443 2.717741169222106
TtestResult(statistic=-0.2705871789378931, pvalue=0.7868673714106337, df=351.499399382814)
ESR_o ESR
20.08367016976556 16.512396694214875
21.052573083699784 15.434302316851127
TtestResult(statistic=3.0820464938083725, pvalue=0.0021858476412679557, df=437.0781486441183)
leukocytes_o leukocytes
7.630994347959347 6.635637860082305
2.4122473129194137 2.1976008482314233
TtestResult(statistic=6.34895562356012, pvalue=6.421621996411517e-10, df=366.10049141658675)
hb_corrected_o hemoglobin
8.46560227591469 8.395797530864199
0.8143255546961468 0.8078859822490667
TtestResult(statistic=1.2297649435019737, pvalue=0.21962177867628688, df=345.5452021878502)
thrombocytes_o thrombocytes
275.04648342764756 275.797520661157
76.76819

In [61]:
for l,t in zip(["das28","Age","sex","RF positive", "aCCP positive","dmard_usage"] # leiden
               ,["das_28_c","Age", "Sex", "RF", "acpa","dmard_usage"] # tacera
              ):
    print(l,t)
    print(baseline_leiden[l].mean(),baseline_tacera[t].mean() )
    print(baseline_leiden[l].std(),baseline_tacera[t].std() )
    print(ttest_ind(baseline_leiden[l],baseline_tacera[t] , equal_var= False))

das28 das_28_c
4.265055320267361 4.954879434450321
1.378825918553534 1.2893384295767447
TtestResult(statistic=-7.536456491792936, pvalue=3.9644143946518837e-13, df=359.3968958094778)
Age Age
59.817299919159254 52.95061728395062
14.727368952516034 14.920091203414689
TtestResult(statistic=6.572788753531747, pvalue=1.8513960167980108e-10, df=341.0572182123902)
sex Sex
0.6443007275666937 0.7283950617283951
0.4789182796169422 0.44570528228101364
TtestResult(statistic=-2.6554203953180577, pvalue=0.008272366298187114, df=360.5937264754427)
RF positive RF
0.46079223928860147 0.9176954732510288
0.49866198285372243 0.27539535416402067
TtestResult(statistic=-20.170193185005136, pvalue=1.3904884050571325e-69, df=604.9837035158826)
aCCP positive acpa
0.46160064672594986 0.8553719008264463
0.4987249362832971 0.3524544438289793
TtestResult(statistic=nan, pvalue=nan, df=nan)
dmard_usage dmard_usage
0.7841552142279709 0.9176954732510288
0.4115735081222808 0.27539535416402067
TtestResult(statistic=-6.30

In [66]:
for l,t in zip(["das28","dmard_usage","Age","sex","RF positive", "aCCP positive"] # leiden
               ,["das_28_c","dmard_usage","Age", "Sex", "RF", "acpa"] # tacera
              ):
    print(l,t)
    print(last_leiden[l].mean(),last_tacera[t].mean() )
    print(last_leiden[l].std(),last_tacera[t].std() )
    print(ttest_ind(last_leiden[l].dropna(),last_tacera[t].dropna(), equal_var= False))

das28 das_28_c
3.005904454901346 3.0485070151654954
1.2888788757154184 1.3512128915465236
TtestResult(statistic=-0.4519046209331777, pvalue=0.6516324486335539, df=332.37896780743426)
dmard_usage dmard_usage
0.7946645109135004 0.9423868312757202
0.40410993907305565 0.23349142690351257
TtestResult(statistic=-7.825182442225053, pvalue=2.4645133863279285e-14, df=571.8257528705241)
Age Age
59.817299919159254 52.95061728395062
14.72736895251602 14.920091203414692
TtestResult(statistic=6.572788753531747, pvalue=1.8513960167980108e-10, df=341.0572182123903)
sex Sex
0.6443007275666937 0.7283950617283951
0.47891827961694244 0.44570528228101325
TtestResult(statistic=-2.6554203953180577, pvalue=0.008272366298187114, df=360.59372647544274)
RF positive RF
0.46079223928860147 0.9176954732510288
0.49866198285372243 0.27539535416402067
TtestResult(statistic=-20.170193185005136, pvalue=1.3904884050571325e-69, df=604.9837035158826)
aCCP positive acpa
0.46160064672594986 0.8553719008264463
0.4987249362832

ValueError: data type <class 'numpy.object_'> not inexact

In [101]:

def print_latex_table_EHR(dataset, 
                          groupby="Subject ID",
                          query = "label==@j" , 
                          type_ = "trajectory", 
                          volgorde= None,
                         features_median = [],
                          features_mean= [],
                          features_add = ["dmard_usage","RF", "acpa", "Sex"]
                         ):
    if query == None:
        query = 'ilevel_0 in ilevel_0'
    print("Features", end ="")
    for i in volgorde:
        print(" & ", end="")
        print(type_,i, end="")
    print("\\\\\\hline")
    print(f"nr of visits (IQR)&"  , end="")
    for j in volgorde:
        
        z = (dataset.query(query)
                        .groupby(groupby)
                        )
        #print(f"n= &{z.head(1).count().iloc[1]}&")
        visits = z.count().iloc[:,1]
        print(f"{int(visits.median())} & " # ({float(round(visits.quantile(q=0.25),1))} - {float(round(visits.quantile(q=0.75),1))})& "
              , end="")
    print("\\\\")
    for i in features_median:
        print(i,"$\space$ (IQR) $\space\dagger$ ", end= "")
        for j in volgorde:
            print(" & ", end="")
            z = (dataset.query(query)
                        .groupby(groupby)
                         [i])
            print(f"{float(round(z.median(),1))} ({float(round(z.quantile(q=0.25),1))} - {float(round(z.quantile(q=0.75),1))})", end="")
        print("\\\\")
    for i in features_mean:
        print(i,"$\space$ (±IQR) $\space\dagger$ ", end= "")
        for j in volgorde:
            print(" & ", end="")
            z = (dataset.query(query)
                        .groupby(groupby)
                        [i])
            print(f"{float(round(z.mean(),2))} ({float(round(z.std(),2))})", end="")
        print("\\\\")
    for i in features_add:
        print(i,"(\\%)", end= "")
        for j in volgorde:
            print(" & ", end="")
            z = (dataset.query(query)
                        .groupby(groupby)
                         [i])
            print(f"{int(round(z.sum(),0))} ({round(float(z.mean()*100), 1)})", end="")

        print("\\\\")



In [90]:
(leiden.query("label == 8" )
                        .groupby("label")
                        ).count()

,TJC_28,SJC_28,ESR,leukocytes,hb_corrected,MCV,thrombocytes,TJC_28_o,SJC_28_o,ESR_o,...,from_clust,global_trajectory_o,global_trajectory,communi_x,communi_y,communi_x.1,communi_y.1,communi,L2,L1
label,,,,,,,,,,,,,,,,,,,,,
8,70,70,70,70,70,70,70,70,70,70,...,70,70,70,70,70,70,70,70,70,70


In [102]:


print_latex_table_EHR(leiden, groupby="label", query='label == @j',type_="label", volgorde =[8,7,6,5,], features_median= features_o + ["das28"], features_mean=  ["Age", "seropos",], features_add=  ["dmard_usage","RF positive", "aCCP positive", "sex"] )

Features & label 8 & label 7 & label 6 & label 5\\\hline
nr of visits (IQR)&70 & 589 & 523 & 485 & \\
TJC_28_o $\space$ (IQR) $\space\dagger$  & 2.5 (1.0 - 5.0) & 11.0 (8.0 - 14.0) & 4.0 (2.0 - 7.0) & 5.0 (3.0 - 6.0)\\
SJC_28_o $\space$ (IQR) $\space\dagger$  & 2.0 (1.0 - 3.0) & 10.0 (8.0 - 13.0) & 3.0 (1.0 - 7.0) & 4.0 (3.0 - 5.0)\\
ESR_o $\space$ (IQR) $\space\dagger$  & 17.0 (9.0 - 32.5) & 29.0 (14.0 - 45.0) & 75.0 (63.0 - 100.5) & 34.0 (29.0 - 40.0)\\
leukocytes_o $\space$ (IQR) $\space\dagger$  & 3.7 (3.1 - 4.4) & 8.5 (6.8 - 10.4) & 9.5 (7.6 - 11.6) & 8.4 (7.2 - 9.9)\\
hb_corrected_o $\space$ (IQR) $\space\dagger$  & 4.6 (4.2 - 4.8) & 8.2 (7.6 - 8.7) & 7.4 (6.8 - 7.9) & 8.3 (7.9 - 8.7)\\
thrombocytes_o $\space$ (IQR) $\space\dagger$  & 265.5 (236.2 - 290.5) & 312.0 (258.0 - 373.0) & 381.0 (298.5 - 462.0) & 312.0 (272.0 - 358.0)\\
MCV_o $\space$ (IQR) $\space\dagger$  & 91.0 (89.0 - 93.0) & 89.0 (86.0 - 92.5) & 89.0 (85.0 - 93.0) & 90.0 (87.0 - 94.0)\\
das28 $\space$ (IQR) $\space\

In [103]:


print_latex_table_EHR(leiden, groupby="label", query='label == @j',type_="label", volgorde =[4,3,2,1], features_median= features_o + ["das28"], features_mean=  ["Age", "seropos",], features_add=  ["dmard_usage","RF positive", "aCCP positive", "sex"] )

Features & label 4 & label 3 & label 2 & label 1\\\hline
nr of visits (IQR)&967 & 674 & 654 & 1055 & \\
TJC_28_o $\space$ (IQR) $\space\dagger$  & 4.0 (3.0 - 7.0) & 1.0 (0.0 - 2.0) & 1.0 (0.0 - 2.0) & 1.0 (0.0 - 1.0)\\
SJC_28_o $\space$ (IQR) $\space\dagger$  & 3.0 (1.0 - 4.0) & 0.0 (0.0 - 1.0) & 0.5 (0.0 - 1.0) & 0.0 (0.0 - 1.0)\\
ESR_o $\space$ (IQR) $\space\dagger$  & 9.0 (6.0 - 14.0) & 11.0 (6.0 - 19.0) & 33.0 (28.0 - 41.0) & 6.0 (2.0 - 11.0)\\
leukocytes_o $\space$ (IQR) $\space\dagger$  & 7.1 (6.1 - 8.3) & 10.3 (9.2 - 12.4) & 7.6 (6.4 - 9.0) & 6.2 (5.2 - 7.2)\\
hb_corrected_o $\space$ (IQR) $\space\dagger$  & 8.7 (8.3 - 9.2) & 8.7 (8.2 - 9.1) & 8.4 (7.9 - 8.7) & 8.7 (8.3 - 9.1)\\
thrombocytes_o $\space$ (IQR) $\space\dagger$  & 259.0 (227.0 - 297.0) & 303.0 (259.2 - 348.0) & 282.0 (240.0 - 334.0) & 241.0 (209.0 - 278.0)\\
MCV_o $\space$ (IQR) $\space\dagger$  & 92.0 (88.0 - 95.0) & 92.0 (88.1 - 95.0) & 92.0 (89.0 - 95.0) & 93.0 (90.0 - 96.0)\\
das28 $\space$ (IQR) $\space\dagger$